REGRESSÃO LINEAR - PARA PRECIFICADOR DE IMÓVEIS

Um dos modelos que será usado para o projeto de precificação de imóveis será o Modelo de Regressão Linear.

A regressão é uma técnica de aprendeixado de máquina supervisionada que é utilizada para prever valores numéricos contínuos, portanto, esse modelo encontra a relação entre as variáveis dependentes (Y/Valor) e independente (X/características) do nosso conjunto de dados de imóveis.

Mais precisamente, utilizaremos será feito uma Regressão Linear Múltipla, pois precismaos encontrar entre as variáveis independentes (X1, X2, ...,Xn (Características)) e a variável dependente (Y/Valor).

Para avaliarmos o Modelo de Regressão utilizaremos algumas metricos, como:

-Root Mean Squere Error (RMSE);

-Mean Square Error (MSE);

-Mean Absolute Error (MAE);

-Median Absolute Error;

-Explained Variance Score;

-R2 Score;

-Adjusted R2.

In [1]:
## Importação das bibliotecas necessárias para o modelo

# Preparar dados
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Métricas Regressão
import sklearn.metrics as sm
import numpy as np

import matplotlib.pyplot as plt

# Criar Modelos
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.layers import Dropout
from keras.regularizers import l1_l2
from keras.callbacks import EarlyStopping

# limpar a tela do jupyter ##Limpa as saidas para deixar o código mais bonito
from IPython.display import clear_output

In [2]:
# Dados Totais
df = pd.read_csv('https://raw.githubusercontent.com/JanotLucas/Projeto-Integrador-I/main/imoveis.csv')

Análise Descritiva dos dados de imóveis

In [3]:
len(df)

48408

In [4]:
df.head()

,index,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,0,SCS Quadra 06 Bloco A Lote 194,7000.0,200.0,7000.0,0,0,0,NaN,2000.0,...,0,0,0,0,0,0,0,0,0,Prédio
1,1,Rua 14,2000.0,63.0,2000.0,2,1,1,86.0,486.0,...,0,0,0,0,0,0,0,0,0,Apartamento
2,2,SQNW 310 Bloco B,5100.0,76.0,5100.0,2,1,2,409.0,791.0,...,0,1,0,0,0,0,0,0,0,Apartamento
3,3,Rua COPAIBA,1800.0,32.0,1800.0,0,0,1,NaN,400.0,...,0,0,0,0,0,0,0,0,0,Sala
4,4,SHIS QL 12,95000.0,1952.0,95000.0,15,0,0,15659.0,NaN,...,0,0,0,0,0,0,0,0,0,Casa


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48408 entries, 0 to 48407
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    48408 non-null  int64  
 1   Título                   48408 non-null  object 
 2   Valor                    48402 non-null  float64
 3   area                     48408 non-null  float64
 4   Valor_metro              48402 non-null  float64
 5   quartos                  48408 non-null  int64  
 6   suites                   48408 non-null  int64  
 7   garagem                  48408 non-null  int64  
 8   iptu                     13934 non-null  float64
 9   condominio               22208 non-null  float64
 10  regiao                   48408 non-null  object 
 11  cidade                   48408 non-null  object 
 12  imobiliaria              48408 non-null  object 
 13  creci                    48408 non-null  object 
 14  operacao              

In [ ]:
# Exibe a porcentagem de valores núlos por coluna
df.isna().sum() * 100 / len(df)

In [ ]:
# Apagando as colunas de IPTU e Condomínio por causa da alta quantidade de valores nulos
df = df.drop(columns=['iptu', 'condominio'])

In [ ]:
# Apagando a coluna 'Valor_metro' porque ela tem o mesmo valor da coluna 'Valor'
df = df.drop(columns=['Valor_metro'])

In [ ]:
#Verificar quais são os nulos na coluna 'Valor'
valores_nulos = df['Valor'].isnull()

# Filtrando para mostrar apenas as linhas com valores nulos na coluna 'Valor'
nulos_na_coluna_valor = df[valores_nulos]
nulos_na_coluna_valor

In [ ]:
# Exibe a porcentagem de valores núlos por coluna
df.isna().sum() * 100 / len(df)

### OBS: Os nulos da coluna 'Valor' e da coluna 'Valor_metro' são os mesmo

In [ ]:
# Pegar a média dos valores por tipo de operação
valor_medio_por_tipo_operacao = df.groupby(['tipo', 'operacao'])['Valor'].mean()

# Formatar os valores para representar duas casas decimais e adiciona separadores de milhares
valor_medio_por_tipo_operacao_formatado = valor_medio_por_tipo_operacao.map('{:,.2f}'.format)

print(valor_medio_por_tipo_operacao_formatado)

In [ ]:
# Filtrar os dados para incluir apenas a região 'Asa Norte'
df_asa_norte = df[df['regiao'] == 'ASA NORTE']

# Calcular a média dos valores por tipo de operação para a região 'Asa Norte'
valor_medio_por_tipo_operacao_asa_norte = df_asa_norte.groupby(['tipo', 'operacao'])['Valor'].mean()

# Formatar os valores para representar duas casas decimais e adicionar separadores de milhares
valor_medio_por_tipo_operacao_asa_norte_formatado = valor_medio_por_tipo_operacao_asa_norte.map('{:,.2f}'.format)

print(valor_medio_por_tipo_operacao_asa_norte_formatado)

### OBS: Após a análise acima, vou substituir os valores nulos pela mediana, pois a média está discrepante.

In [ ]:
# Calcular a mediana para cada combinação de tipo de imóvel e região
mediana_por_tipo_e_regiao = df.groupby(['tipo', 'regiao'])['Valor'].median()

# Substituir os valores nulos pela mediana correspondente de cada tipo de imóvel e região
for tipo, regiao in mediana_por_tipo_e_regiao.index:
    filtro = (df['tipo'] == tipo) & (df['regiao'] == regiao)
    mediana = mediana_por_tipo_e_regiao[(tipo, regiao)]
    df.loc[filtro, 'Valor'] = df.loc[filtro, 'Valor'].fillna(mediana)

In [ ]:
# Exibe a porcentagem de valores núlos por coluna
df.isna().sum() * 100 / len(df)

In [ ]:
# Resumo estatístico básico
resumo_df = df.describe()
resumo_df

Após a análise descritiva dos dados, será feito a separação da base de dados em imóveis que estão para venda e para aluguel, para assim, fazer o modelo de cada um.

Análise Descritiva dos dados de imóveis para a venda

In [ ]:
#Base Venda
df_venda = df[df['operacao'] == 'venda']

In [ ]:
df_venda.head()

In [ ]:
len(df_venda)

In [ ]:
df_venda.info()

Removendo as colunas que eu considero irrelevante para o modelo

In [ ]:
df_venda = df_venda.drop(columns=[ 'index', 'Título', 'cidade', 'imobiliaria', 'creci', 'operacao', 'Aquecimento Solar', 'Circuito de TV', 'Gerador de Energia', 'Cobertura Coletiva', 'Guarita', 'Interfone', 'Mobiliado', 'Despensa', 'Escritório', 'Gás Canalizado', 'Jardim', 'Lavabo', 'Lazer no Pilotis',  'Poço Artesiano', 'Portão Eletrônico', 'Quadra Esportiva', 'Sala de Jogos', 'Salão de Jogos', 'Salão Gourmet', 'Sauna', 'SPA', 'Área de Serviço Coberta', 'Piso em Porcelanato', 'Vista Livre', 'Home Theater', 'Aceita Pet' ])

In [ ]:
df_venda.info()

In [ ]:
df_venda['tipo'].value_counts()

In [ ]:
# Configurar para exibir todas as linhas
pd.set_option('display.max_rows', None)

In [ ]:
df_venda['regiao'].value_counts()

Transformando as string em numérico

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Criar uma instância do LabelEncoder
label_encoder = LabelEncoder()

# Ajustar o encoder aos valores da coluna região
df_venda['regiao_encoded'] = label_encoder.fit_transform(df_venda['regiao'])

# Mapear os valores originais para os valores codificados
mapeamento = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Mostrar o mapeamento
print("Mapeamento:")
for regiao, encoded_value in mapeamento.items():
    print(f"{regiao}: {encoded_value}")

In [ ]:

# Ajustar o encoder aos valores da coluna região
df_venda['tipo_encoded'] = label_encoder.fit_transform(df_venda['tipo'])

# Mapear os valores originais para os valores codificados
mapeamento = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Mostrar o mapeamento
print("Mapeamento:")
for tipo, encoded_value in mapeamento.items():
    print(f"{tipo}: {encoded_value}")

Separar o X e o y, Normalizando os dados, e separar os dados em treinamento, teste e validação

In [ ]:
df_venda = df_venda.drop(columns=['regiao', 'tipo'])

In [ ]:
df_venda.info()

In [ ]:
# Separar X e y
X_regressao = df_venda.drop('Valor', axis = 1)
y_regressao = df_venda[['Valor']]

# Separar X e y em treinamento (80%), validação (10%) e teste (10%)
X_train_regressao, X_temp, y_train_regressao, y_temp = train_test_split(X_regressao, y_regressao, test_size=0.2, random_state=42)
X_val_regressao, X_test_regressao, y_val_regressao, y_test_regressao = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalizar X e y
scaler_X_regressao = MinMaxScaler()
X_regressao = scaler_X_regressao.fit_transform(X_regressao)

scaler_y_regressao = MinMaxScaler()
y_regressao = scaler_y_regressao.fit_transform(y_regressao)


# Exibe as 5 primeiro linhas do df_venda
df_venda.head()

Criar modelo e Treinar o modelo

In [ ]:
# Criar modelo
model_regressao = Sequential()
model_regressao.add(Input(shape=(X_train_regressao.shape[1])))  # Camada de entrada

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(1, activation='linear'))  # Saída linear para regressão

# Compilar modelo
model_regressao.compile(loss='mean_squared_error', optimizer='adam')

# Configurar Early Stopping ##Depois de 10 epochas o modelo não melhorar vai parar de treinar. Verbose=1 vai mostrar as epochas.
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True)

# Treinar modelo com o Early Stopping como callback.
history = model_regressao.fit(X_train_regressao,
                    y_train_regressao,
                    epochs=40,
                    batch_size=32,
                    validation_data=(X_val_regressao, y_val_regressao),
                    callbacks=[early_stopping])


clear_output(wait=True)


def plot_loss(history):
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Treinamento')
    plt.plot(history.history['val_loss'], label='Validação')
    plt.title('Loss ao Longo das Épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_loss(history)

Metricas e Avaliação do Modelo

In [ ]:
def metricas_regressao(X_test, y_test, scaler, model):
    """
    Avalia métricas de regressão para um modelo e conjunto de teste fornecidos.

    Parâmetros:
    - X_test: características do conjunto de teste.
    - y_test: rótulos verdadeiros do conjunto de teste.
    - scaler_y: scaler utilizado para normalizar a variável alvo.
    - model: modelo treinado para fazer previsões.

    Retorna:
    Métricas de avaliação de regressão impressas.
    """

    # 1. Fazer previsões usando o modelo fornecido
    predict = model.predict(X_test)

    # 2. Inverter a transformação para obter os valores originais (não normalizados)
    predict = scaler_y_regressao.inverse_transform(predict)
    real = scaler_y_regressao.inverse_transform(y_test)

    # 3. Calcular R2 e R2 ajustado
    k = X_test.shape[1]
    n = len(X_test)
    r2 = sm.r2_score(real, predict)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)  # fórmula para R2 ajustado


    # 4. Imprimir métricas
    print('Root Mean Square Error:', round(np.sqrt(np.mean(np.array(predict) - np.array(real))**2), 2))
    print('Mean Square Error:', round(sm.mean_squared_error(real, predict), 2))
    print('Mean Absolut Error:', round(sm.mean_absolute_error(real, predict), 2))
    print('Median Absolut Error:', round(sm.median_absolute_error(real, predict), 2))
    print('Explain Variance Score:', round(sm.explained_variance_score(real, predict) * 100, 2))
    print('R2 score:', round(sm.r2_score(real, predict) * 100, 2))
    print('Adjusted R2 =', round(adj_r2, 3) * 100)

In [ ]:
metricas_regressao(X_test_regressao, y_test_regressao, scaler_y_regressao, model_regressao)

In [ ]:
predict = model_regressao.predict(X_test_regressao)

# Desnormaliza o y_test na variável y_desnormalizado
y_test_regressao_desnormalizado = scaler_y_regressao.inverse_transform(y_test_regressao)

# Desnormaliza o predict na variável predict_desnormalizado
predict_desnormalizado = scaler_y_regressao.inverse_transform(predict)

#Gráfico
grafico_x = [x for x in range(1,len(y_test_regressao_desnormalizado)+1)]

plt.plot(grafico_x, y_test_regressao_desnormalizado, label='Real')
plt.plot(grafico_x, predict_desnormalizado, label='Predito')

plt.title('Comparando valores reais e preditos de teste')
plt.legend()

plt.ylabel('Valor')

plt.show()

Análise Descritiva dos dados de imóveis para a Aluguel

In [ ]:
#Base Venda
df_aluguel = df[df['operacao'] == 'aluguel']

In [ ]:
df_venda.head()

In [ ]:
len(df_venda)

In [ ]:
df_venda.info()

Removendo as colunas que eu considero irrelevante para o modelo

In [ ]:
df_venda = df_venda.drop(columns=[ 'index', 'Título', 'cidade', 'imobiliaria', 'creci', 'operacao', 'Aquecimento Solar', 'Circuito de TV', 'Gerador de Energia', 'Cobertura Coletiva', 'Guarita', 'Interfone', 'Mobiliado', 'Despensa', 'Escritório', 'Gás Canalizado', 'Jardim', 'Lavabo', 'Lazer no Pilotis',  'Poço Artesiano', 'Portão Eletrônico', 'Quadra Esportiva', 'Sala de Jogos', 'Salão de Jogos', 'Salão Gourmet', 'Sauna', 'SPA', 'Área de Serviço Coberta', 'Piso em Porcelanato', 'Vista Livre', 'Home Theater', 'Aceita Pet' ])
df_venda

In [ ]:
df_venda.info()

In [ ]:
df_venda['tipo'].value_counts()

In [ ]:
# Configurar para exibir todas as linhas
pd.set_option('display.max_rows', None)

In [ ]:
df_venda['regiao'].value_counts()

Transformando as string em numérico

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Criar uma instância do LabelEncoder
label_encoder = LabelEncoder()

# Ajustar o encoder aos valores da coluna região
df_venda['regiao_encoded'] = label_encoder.fit_transform(df_venda['regiao'])

# Mapear os valores originais para os valores codificados
mapeamento = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Mostrar o mapeamento
print("Mapeamento:")
for regiao, encoded_value in mapeamento.items():
    print(f"{regiao}: {encoded_value}")

In [ ]:
# Ajustar o encoder aos valores da coluna região
df_venda['tipo_encoded'] = label_encoder.fit_transform(df_venda['tipo'])

# Mapear os valores originais para os valores codificados
mapeamento = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Mostrar o mapeamento
print("Mapeamento:")
for tipo, encoded_value in mapeamento.items():
    print(f"{tipo}: {encoded_value}")

Separar o X e o y, Normalizando os dados, e separar os dados em treinamento, teste e validação

In [ ]:
df_venda = df_venda.drop(columns=['regiao', 'tipo'])

In [ ]:
# Separar X e y
X_regressao = df_venda.drop('Valor', axis=1)
y_regressao = df_venda['Valor']

# Separar X e y em treinamento (80%), validação (10%) e teste (10%)
X_train_regressao, X_temp, y_train_regressao, y_temp = train_test_split(X_regressao, y_regressao, test_size=0.2, random_state=42)
X_val_regressao, X_test_regressao, y_val_regressao, y_test_regressao = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalizar X apenas
scaler = MinMaxScaler()
X_train_regressao = scaler.fit_transform(X_train_regressao)
X_val_regressao = scaler.transform(X_val_regressao)
X_test_regressao = scaler.transform(X_test_regressao)

# Exibe as 5 primeiro linhas do df_venda
df_venda.head()

Criar modelo e Treinar o modelo

In [ ]:
# Criar modelo
model_regressao = Sequential()
model_regressao.add(Input(shape=(X_train_regressao.shape[1])))  # Camada de entrada

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(50, activation='relu'))

model_regressao.add(Dense(1, activation='linear'))  # Saída linear para regressão

# Compilar modelo
model_regressao.compile(loss='mean_squared_error', optimizer='adam')

# Configurar Early Stopping ##Depois de 10 epochas o modelo não melhorar vai parar de treinar. Verbose=1 vai mostrar as epochas.
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True)

# Treinar modelo com o Early Stopping como callback.
history = model_regressao.fit(X_train_regressao,
                    y_train_regressao,
                    epochs=40,
                    batch_size=32,
                    validation_data=(X_val_regressao, y_val_regressao),
                    callbacks=[early_stopping])


clear_output(wait=True)


def plot_loss(history):
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Treinamento')
    plt.plot(history.history['val_loss'], label='Validação')
    plt.title('Loss ao Longo das Épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_loss(history)

Metricas e Avaliação do Modelo

In [ ]:
def metricas_regressao(X_test, y_test, scaler, model):
    """
    Avalia métricas de regressão para um modelo e conjunto de teste fornecidos.

    Parâmetros:
    - X_test: características do conjunto de teste.
    - y_test: rótulos verdadeiros do conjunto de teste.
    - scaler_y: scaler utilizado para normalizar a variável alvo.
    - model: modelo treinado para fazer previsões.

    Retorna:
    Métricas de avaliação de regressão impressas.
    """

    # 1. Fazer previsões usando o modelo fornecido
    predict = model.predict(X_test)

    # 2. Inverter a transformação para obter os valores originais (não normalizados)
    predict = scaler_y.inverse_transform(predict)
    real = scaler_y.inverse_transform(y_test)

    # 3. Calcular R2 e R2 ajustado
    k = X_test.shape[1]
    n = len(X_test)
    r2 = sm.r2_score(real, predict)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)  # fórmula para R2 ajustado


    # 4. Imprimir métricas
    print('Root Mean Square Error:', round(np.sqrt(np.mean(np.array(predict) - np.array(real))**2), 2))
    print('Mean Square Error:', round(sm.mean_squared_error(real, predict), 2))
    print('Mean Absolut Error:', round(sm.mean_absolute_error(real, predict), 2))
    print('Median Absolut Error:', round(sm.median_absolute_error(real, predict), 2))
    print('Explain Variance Score:', round(sm.explained_variance_score(real, predict) * 100, 2))
    print('R2 score:', round(sm.r2_score(real, predict) * 100, 2))
    print('Adjusted R2 =', round(adj_r2, 3) * 100)

In [ ]:
metricas_regressao(X_test_regressao, y_test_regressao, scaler, model_regressao)

In [ ]:
#Gráfico
grafico_x = [x for x in range(1,len(y_test_regressao_desnormalizado)+1)]

plt.plot(grafico_x, y_test_regressao_desnormalizado, label='Real')
plt.plot(grafico_x, predict_desnormalizado, label='Predito')

plt.title('Comparando valores reais e preditos de teste')
plt.legend()

plt.ylabel('Valor')

plt.show()